In [1]:
import pandas as pd
import numpy as np
import glob

#data import
#from dataExtraction import *

#LIBSVM
from libsvm.svmutil import *

#nilearn imports
import nilearn
from nilearn import plotting, image, interfaces, maskers
from nilearn.image import mean_img
from nilearn.plotting import plot_anat, plot_img, plot_stat_map, show, plot_design_matrix
from nilearn.glm import threshold_stats_img
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn.reporting import get_clusters_table
import nibabel as nib
from nilearn.maskers import NiftiMasker

#sklearn imports
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:
def load_beta_data(topSubjects, data_type):
    taskType = ['colorwheel', 'samedifferent']
    num_runs = [1, 2, 3, 4]
    
    X, y = [], []

    for subjID in topSubjects:
        for task in taskType:
            for run in num_runs:
                try:
                    file_path = f"~/teams/a05/group_1_data/{data_type}Beta/beta_{subjID}_{task}_{run}.nii.gz"
                    
                    beta_img = nib.load(str(file_path))
                    
                    # getting the data as an array, then flattening to 1D feature vector for model training
                    mask_img = nilearn.datasets.load_mni152_brain_mask(resolution=2, threshold=0.2)
                    masker = NiftiMasker(mask_img=mask_img, memory="nilearn_cache", memory_level=1).fit()

                    #transform all the data to get the brain voxels
                    beta_data = masker.transform(beta_img).flatten()
                
                    X.append(beta_data)
    
                    # appending the task category to y
                    y.append(task)
                except:
                    #if there isn't a specific run, i.e. run 4
                    continue

    return np.array(X), y

In [8]:
def train_svmLight(X, y):
    mask_img = nilearn.datasets.load_mni152_brain_mask(resolution=2, threshold=0.2)
    masker = NiftiMasker(mask_img=mask_img, memory="nilearn_cache", memory_level=1).fit()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    # SVM classifier
    svm_model = svm_train(y_train, X_train, '-s 3 -t 2 -c 1 -g 0.1')
    svm_save_model('svr_beta_model', svm_model)
    y_pred, accuracy, _ = svm_predict(y_test, X_test, svm_model)
    

    # Evaluate performance
    print("Model Accuracy: " + str(accuracy))

    # Confusion matrix
    #cm = confusion_matrix(y_test, y_pred)
    #disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    #disp.plot()
    #show()

    #print 3D model of predicted brain
    svm_model = svm_load_model('svr_beta_model')
    #select random subject
    predicted_index = np.random.choice(len(y_test))
    predicted_beta, _, _ = svm_predict([y_test[predicted_index]], [X_test[predicted_index]], svm_model)
    weights_3d = masker.inverse_transform(predicted_beta)
    plotting.plot_stat_map(weights_3d, vmax=2, alpha=0.5, title=f"Predicted Brain")
    plotting.plot_stat_map(X_test[predicted_index], vmax=2, alpha=0.5, title=f"Actual Brain")

    return svm_model

In [9]:
subjects = [103, 105, 106, 110, 112, 113, 115, 124, 127, 130, 
            131, 133, 138, 142, 143, 145, 157, 159, 161, 165, 
            173, 176, 177, 183, 187, 195, 200, 207, 208, 109,
            117, 140, 147, 172, 178, 180, 181, 182, 188]

X, y = load_beta_data(subjects, 'nonConfound')
y = [1 if task == 'colorwheel' else 0 for task in y]

In [10]:
train_svmLight(X, y)

*.*
optimization finished, #iter = 494
nu = 0.399638
obj = -17.274058, rho = -0.492593
nSV = 216, nBSV = 0
Mean squared error = 0.250459 (regression)
Squared correlation coefficient = nan (regression)
Model Accuracy: (0.0, 0.2504589100885396, nan)
Mean squared error = 0.242647 (regression)
Squared correlation coefficient = nan (regression)


TypeError: X must be of shape (samples, 235375).

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
svm_model = svm_load_model('svr_beta_model')
mask_img = nilearn.datasets.load_mni152_brain_mask(resolution=2, threshold=0.2)
masker = NiftiMasker(mask_img=mask_img, memory="nilearn_cache", memory_level=1).fit()
#select random subject
predicted_index = np.random.choice(len(y_test))
predicted_beta, _, _ = svm_predict([y_test[predicted_index]], [X_test[predicted_index]], svm_model)
print(predicted_beta)

sv_coef = np.array(svm_model.get_sv_coef()).flatten()  # Coefficients (alpha * y)
support_vectors = np.array([list(sv.values()) for sv in svm_model.get_SV()])  # Support vectors

# Compute w (only valid for linear kernel)
w = np.sum(sv_coef[:, np.newaxis] * support_vectors, axis=0)
b = -model.rho[0]  # Bias term

# Target prediction value (e.g., predicted decision function output)
y_target = y_test[predicted_index]  # Adjust this based on expected output

# Solve for x using pseudo-inverse (Moore-Penrose)
x_predicted = pinv(w) @ (y_target - b)

weights_3d = masker.inverse_transform(x_predicted)
plotting.plot_stat_map(weights_3d, vmax=2, alpha=0.5, title=f"Predicted Brain")
plotting.plot_stat_map(X_test[predicted_index], vmax=2, alpha=0.5, title=f"Actual Brain")

Mean squared error = 0.257462 (regression)
Squared correlation coefficient = nan (regression)
[0.4925925925925935]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (216,) + inhomogeneous part.

In [ ]:
X1, y1 = load_beta_data(subjects, 'confound')
y1 = [1 if task == 'colorwheel' else 0 for task in y1]

In [ ]:
train_svmLight(X1, y1)

In [ ]:
help(svm_predict)

In [ ]:
help(svm_train)